In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import ShuffleSplit, KFold
import matplotlib.pyplot as plt
from scipy import sparse
from itertools import product
from time import time
import seaborn
import pickle
from collections import Counter
import heapq
import pandas as pd

In [2]:
#load everything
num_ing = 8023
num_items = 8576
num_rec = 178265
recipe_decoder = pickle.load( open('data-cleaned/recipe_decoder.pkl', 'rb') )
ingr_decoder   = pickle.load( open('data-cleaned/ingredient_decoder.pkl', 'rb') )
tag_decoder    = pickle.load( open('data-cleaned/tag_decoder.pkl', 'rb') )
R     = sparse.load_npz("data-cleaned/recipes.npz")
Rhat  = sparse.load_npz("data-cleaned/recipes_tfidf.npz")
U     = sparse.load_npz("data-cleaned/user_train.npz")
Uhat  = sparse.load_npz("data-cleaned/user_train_tfidf.npz")
Utest = sparse.load_npz("data-cleaned/user_test.npz")
col_sum = np.load("data-cleaned/recipe_col_sum.npy")
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [3]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from recipeClasses import *
from userClasses import *

In [4]:
#prepare testing data and splitting data
user_test = Utest[:,-1].toarray().flatten().astype('int')
y = np.zeros((user_test.shape[0], 2), dtype='int')-1
for i in range(len(y)):
    recipes = Utest[i].nonzero()[1][:-1]
    if len(recipes) == 1:
        y[i,0] = recipes
    elif len(recipes) == 2:
        y[i,:] = recipes
    else:
        raise ValueError("Someone reviewed 3 recipes!")

Urecipe = user_to_recipe(U[user_test], 4)
y = y

In [7]:
pipe = Pipeline([("dr", TruncatedSVD(n_components=10)),
                    ("rdr", userCluster(sc='int', n_clusters=50))])

pipe.fit(U)
pipe.score(U[user_test][:1000], y[:1000])
# p = pipe.predict(U[user_test][:50])

8.743799999999998

In [7]:
pipe = Pipeline([("dr", TruncatedSVD(n_components=10)),
                    ("rdr", recipeKNN(sc='com'))])

pipe.fit(R)
pipe.score(Urecipe, y)

-58.093999999999994

In [ ]:
columns = ["kNN", "NNBall", "KMeans", "GMM", "MinCut", "KMeansNN", "GMMNN", "MinCutNN", ]
rows = ["PCA", "KPCA", "NMF", "LDA"]
d = pd.DataFrame(tuple(), columns=columns, index=rows)
# d.to_pickle("results/user_U_sum.pkl")
# d.to_pickle("results/user_U_fr.pkl")
# d.to_pickle("results/user_U_rf.pkl")
# d.to_pickle("results/user_Uhat_sum.pkl")
# d.to_pickle("results/user_Uhat_fr.pkl")
# d.to_pickle("results/user_Uhat_rf.pkl")
d.to_pickle("results/recipe_R_int.pkl")
d.to_pickle("results/recipe_R_com.pkl")
d.to_pickle("results/recipe_Rhat_int.pkl")
d.to_pickle("results/recipe_Rhat_com.pkl")

In [ ]:
columns = [1, 2, 3, 4, 5]
rows = ["PCA", "KPCA", "NMF", "LDA"]
d = pd.DataFrame(tuple(), columns=columns, index=rows)
d.to_pickle("results/recipe_R_int.pkl")
d.to_pickle("results/recipe_R_com.pkl")
d.to_pickle("results/recipe_Rhat_int.pkl")
d.to_pickle("results/recipe_Rhat_com.pkl")

In [6]:
X = np.arange(10).reshape(5,2)
searched_values = np.array([[0,1],[4,5]])
npi.indices(X, searched_values)

array([0, 2])

In [28]:
pca = TruncatedSVD(n_components=10)
upca = pca.fit_transform(U)
some = pca.transform(U[user_test])

In [29]:
user_test

array([    2,     5,    16, ..., 25070, 25071, 25074])

In [30]:
indices = []
i = 0
j = 0
while i < upca.shape[0]:
    if np.allclose(upca[i], some[j]):
        indices.append(i)
        j += 1
    if j >= some.shape[0]:
        break
    i += 1
indices = np.array(indices)

In [31]:
indices

array([    2,     5,    16, ..., 25070, 25071, 25074])